In [1]:
import numpy as np
from scipy import linalg, signal
import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True)
plt.rcParams.update({
    'font.family':'Times New Roman',
    # 'text.usetex': True
})

## AR(1)

In [2]:
# tf with control
import control
a = -0.5
H1 = control.tf([1, 0], [1, a], 1)
H1inv =  control.tf([1, 0], [a ,1], 1)
H = control.series(H1, H1inv)
display(H)

TransferFunction(array([1., 0., 0.]), array([-0.5 ,  1.25, -0.5 ]), 1)

In [3]:
# correlation 
from sympy.abc import a, s
aa = np.convolve(([a, 1]),[1, a])
b = [1]
from sympy.abc import z
num = sum([b[q]*z**(-q) for q in range(len(b))])
den = sum([aa[q]*z**(-q) for q in range(len(aa))])*z
P = num/den
from sympy.simplify import simplify
display(simplify(P))
P0 = simplify(P/z*(z+a)*s**2)

def myitr(max):
    n, P = 0, P0
    while n < max:
        if n==0:
            yield P.subs({z: -a}).subs(a, -0.6894)+ 0.1  # measurement noise
        else:
            yield P.subs({z: -a}).subs(a, -0.6894)

        P *= z
        n  += 1
    return 'done'

p = myitr(8)
while 1:
    try:
        display(next(p))
    except StopIteration as e:
        print('Generator return value:', e.value)
        break

z/(a*z**2 + a + z*(a**2 + 1))

1.90575057185857*s**2 + 0.1

1.3138244442393*s**2

0.905750571858574*s**2

0.624424444239301*s**2

0.430478211858574*s**2

0.296771679255301*s**2

0.204594395678604*s**2

0.14104737638083*s**2

Generator return value: done


## Example 3.1

In [4]:
# tf with control
import control
Hin = control.tf([0, 1], [1, -0.5], 1)
Hin_inv = control.tf([1, 0], [-0.5, 1], 1)
display(Hin)
display(Hin_inv)
Hc =  control.tf([0, 1], [1, 0.8], 1)
Hc_inv = control.tf([0, 1][::-1], [1, 0.8][::-1], 1)
display(Hc)
display(Hc_inv)
display(control.tf([1], [1, 0], 1))
Hdx1 = control.series(Hin, Hin_inv, Hc_inv, control.tf([1], [1, 0], 1))
display(Hdx1)

TransferFunction(array([1]), array([ 1. , -0.5]), 1)

TransferFunction(array([1, 0]), array([-0.5,  1. ]), 1)

TransferFunction(array([1]), array([1. , 0.8]), 1)

TransferFunction(array([1, 0]), array([0.8, 1. ]), 1)

TransferFunction(array([1]), array([1, 0]), 1)

TransferFunction(array([1., 0., 0.]), array([-0.4 ,  0.5 ,  0.85, -0.5 ,  0.  ]), 1)

In [5]:
# remove same zeros or poles, if zero
def findzp(zeros, poles):
    if not isinstance(zeros, list):
        z = zeros.tolist()
        p = poles.tolist()
    common = np.intersect1d(z, p)
    while common.shape[0]:
        for item in common:
            z.remove(item)
            p.remove(item)
        common = np.intersect1d(z, p)
    return z,p

# test function
zeros = np.array([.0, .0, .3, .5, .5, .6, .6])
poles = np.array([.0, .5, .5, .4, .5, .7, .6, .6])
# den = np.array([.1, .2, .4, .7, .7, .7, .7, .7, .9])
zeros,poles = findzp(zeros, poles)
print(zeros,poles)
pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)

[0.0, 0.3] [0.4, 0.5, 0.7]
[0 1 2]
[0.4, 0.5, 0.7]


In [6]:
# todo: 二级极点
from functools import reduce
from sympy import diff

def prod(x, y):
    return x * (z - y)

def correlation_double_side(max):
    if len(set(poles)) == len(poles):
        pass
    else:
        raise FooError('order too high: must be one')
    n = 0
    p0 = 0
    P0 = []
    for index in pi:
        pl = poles[:]
        pl.pop(index)
        if poles[index] == 0:
            Pl = reduce(prod, [1] + zeros) / reduce(prod, [1] + pl)
            Pl = diff(Pl)
        else:
            Pl = reduce(prod, [1] + zeros) / reduce(prod, [1] + pl) / z
        display(simplify(Pl))
        P0.append(Pl)
        p0+= Pl.subs({z: poles[index]}).evalf()
    P = P0
    p = p0
    while n < max:
        yield p*k
        p = 0
        P = [Pl*z for Pl in P]
        p = sum(Pl.subs({z: poles[index]}).evalf() for Pl,index in zip(P,pi))
        n  += 1
    return 'done'

In [7]:
# Hdx1
zeros, poles, k = signal.tf2zpk(Hdx1.num[0][0], Hdx1.den[0][0])
zeros,poles = findzp(zeros, poles)
print(zeros,poles,k)

pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)


p = correlation_double_side(8)
while 1:
    try:
        print(next(p))
    except StopIteration as e:
        print('Generator return value:', e.value)
        break

[0.0] [2.0, -1.2500000000000002, 0.5] -2.5
[2]
[0.5]


1/((z - 2.0)*(z + 1.25))

0.952380952380952
0.476190476190476
0.238095238095238
0.119047619047619
0.0595238095238095
0.0297619047619048
0.0148809523809524
0.00744047619047619
Generator return value: done


In [8]:
# Hx1x1
Hx1x1 = control.series(Hin, Hin_inv, Hc, Hc_inv,)
display(Hx1x1)

zeros, poles, k = signal.tf2zpk(Hx1x1.num[0][0], Hx1x1.den[0][0])
zeros,poles = findzp(zeros, poles)
print(zeros,poles,k)

pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)
p = correlation_double_side(8)
rxx = []
while 1:
    try:
        rxx.append(next(p))
        print(rxx[-1])
    except StopIteration as e:
        print('Generator return value:', e.value)
        break
rxx[0] += 0.1  # measurement noise

TransferFunction(array([1., 0., 0.]), array([-0.4 ,  0.18,  1.25,  0.18, -0.4 ]), 1)

[0.0, 0.0] [1.9999999999999947, -1.2499999999999996, -0.7999999999999995, 0.4999999999999999] -2.5
[2 3]
[-0.7999999999999995, 0.4999999999999999]


z/((z - 1.99999999999999)*(z - 0.5)*(z + 1.25))

z/((z - 1.99999999999999)*(z + 0.799999999999999)*(z + 1.25))

1.58730158730159
-0.793650793650794
0.873015873015874
-0.579365079365079
0.523015873015873
-0.388650793650793
0.325801587301587
-0.253200793650793
Generator return value: done


In [9]:
# Hdd
Hdd = control.series(Hin, Hin_inv,)
display(Hdd)

zeros, poles, k = signal.tf2zpk(Hdd.num[0][0], Hdd.den[0][0])
zeros,poles = findzp(zeros, poles)
print(zeros,poles,k)

pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)
p = correlation_double_side(8)
rdd = []
while 1:
    try:
        rdd.append(next(p))
        print(rdd[-1])
    except StopIteration as e:
        print('Generator return value:', e.value)
        break

TransferFunction(array([1., 0.]), array([-0.5 ,  1.25, -0.5 ]), 1)

[0.0] [2.0, 0.5] -2.0
[1]
[0.5]


1/(z - 2.0)

1.33333333333333
0.666666666666667
0.333333333333333
0.166666666666667
0.0833333333333333
0.0416666666666667
0.0208333333333333
0.0104166666666667
Generator return value: done


## Problem 3.12

In [10]:
# parameters, correlation by hand using closed method
a1, a2 = 1.2, 0.81
pi = np.pi
r = np.sqrt(a2)
f0 = np.arccos(-a1/(2*r))/(2*pi)
phi = np.arctan((1-r**2)/(1+r**2)/np.tan(2*pi*f0))
poles = r*np.exp(1j*2*pi*f0)
print(np.asarray([poles, poles.conj()]))
rxx = [(1+r**2)/(1-r**2)*np.sqrt(1+(1-r**2)/(1+r**2)**2/np.tan(2*pi*f0))*r**k*np.cos(2*pi*f0*k-phi)/(1-2*r**2*np.cos(4*pi*f0)+r**4) for k in range(5)]
rxx = [rxi/rxx[0] for rxi in rxx]
rxx[0] += 0.04
# print(['%.3f' % r for r in rxx])
print(np.asarray(rxx))

[-0.6+0.671j -0.6-0.671j]
[ 1.04  -0.663 -0.014  0.554 -0.654]


In [11]:
# zeros, poles, k with tf
a1, a2 = 1.2, 0.81
zeros, poles, k = signal.tf2zpk([1, 0, 0], np.convolve(([1 ,a1 , a2]),[a2, a1, 1]))
print(np.asarray(zeros), '\n', np.asarray(poles), '\n', '%.3f' % k)
print('%.3f' % (1/a2,))

[0. 0.] 
 [-0.741+0.828j -0.741-0.828j -0.6  +0.671j -0.6  -0.671j] 
 1.235
1.235


In [12]:
# poles with control
import control
Hin = control.tf([1, 0, 0], [1 ,a1, a2], 1)
Hin_inv = control.tf([1, 0, 0][::-1], [1 , a1, a2][::-1], 1)
# num = [.1, .3, .0, -0.2, -0.4, -0.7, -0.4, -0.2]
num = [1.0, 0.9, 0.1, 0.2]
den = [1]+ [0]*(len(num)-1)
Hfir = control.tf(num, den, 1)
Hfir_inv = control.tf(num[::-1], den[::-1], 1)
display(Hin)
display(Hin_inv)
display(Hfir)
display(Hfir_inv)
pole = control.pole(Hin)
print(pole)
print(abs(pole))

TransferFunction(array([1, 0, 0]), array([1.  , 1.2 , 0.81]), 1)

TransferFunction(array([1]), array([0.81, 1.2 , 1.  ]), 1)

TransferFunction(array([1. , 0.9, 0.1, 0.2]), array([1, 0, 0, 0]), 1)

TransferFunction(array([0.2, 0.1, 0.9, 1. ]), array([1]), 1)

[-0.6+0.671j -0.6-0.671j]
[0.9 0.9]


In [13]:
Hx1x1 = control.series(Hin, Hin_inv)
display(Hx1x1)

TransferFunction(array([1., 0., 0.]), array([0.81 , 2.172, 3.096, 2.172, 0.81 ]), 1)

In [14]:
# Hx1x1
zeros, poles, k = signal.tf2zpk(Hx1x1.num[0][0], Hx1x1.den[0][0])
zeros,poles = findzp(zeros, poles)
print(np.asarray(zeros))

print(np.asarray(poles))

pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)

rxx = []
p = correlation_double_side(8)
while 1:
    try:
        rxx.append(next(p))
        # print(rxx[-1])
    except StopIteration as e:
        print('Generator return value:', e.value)
        break
sigma2_x = rxx[0]  # 应当为倒数，就不改了
rxx = [rxi/rxx[0] for rxi in rxx]
rxx[0] +=0.04
print(np.asarray(rxx))

[0. 0.]
[-0.741+0.828j -0.741-0.828j -0.6  +0.671j -0.6  -0.671j]
[2 3]
[(-0.5999999999999995+0.6708203932499363j), (-0.5999999999999995-0.6708203932499363j)]


z/((z + 0.6 + 0.670820393249936*I)*(z + 0.740740740740741 - 0.828173324999924*I)*(z + 0.740740740740741 + 0.828173324999924*I))

z/((z + 0.6 - 0.670820393249936*I)*(z + 0.740740740740741 - 0.828173324999924*I)*(z + 0.740740740740741 + 0.828173324999924*I))

Generator return value: done
[1.04000000000000 -0.662983425414368 -0.0144198895027580 0.554320441988946
 -0.653504419889501 0.335205745856355 0.127091685082869 -0.424026676243090]


In [15]:
Hdx1 = control.series(Hin, Hin_inv, Hfir)
display(Hdx1)

TransferFunction(array([1. , 0.9, 0.1, 0.2, 0. , 0. ]), array([0.81 , 2.172, 3.096, 2.172, 0.81 , 0.   , 0.   , 0.   ]), 1)

In [16]:
# Hdx1
zeros, poles, k = signal.tf2zpk(Hdx1.num[0][0], Hdx1.den[0][0])
zeros,poles = findzp(zeros, poles)
print(np.asarray(zeros))

print(np.asarray(poles))

pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)

rdx = []
p = correlation_double_side(8)
while 1:
    try:
        rdx.append(next(p) / sigma2_x)
        # print(rdx[-1])
    except StopIteration as e:
        print('Generator return value:', e.value)
        break
print(np.asarray(rdx))

[-1.  +0.j     0.05+0.444j  0.05-0.444j]
[-0.741+0.828j -0.741-0.828j -0.6  +0.671j -0.6  -0.671j  0.   +0.j   ]
[2 3 4]
[(-0.5999999999999995+0.6708203932499363j), (-0.5999999999999995-0.6708203932499363j), 0j]


-(z + 1.0)*(-z + 0.0500000000000001 + 0.44440972086578*I)*(z - 0.0500000000000001 + 0.44440972086578*I)/(z**2*(z + 0.6 + 0.670820393249936*I)*(z + 0.740740740740741 - 0.828173324999924*I)*(z + 0.740740740740741 + 0.828173324999924*I))

-(z + 1.0)*(-z + 0.0500000000000001 + 0.44440972086578*I)*(z - 0.0500000000000001 + 0.44440972086578*I)/(z**2*(z + 0.6 - 0.670820393249936*I)*(z + 0.740740740740741 - 0.828173324999924*I)*(z + 0.740740740740741 + 0.828173324999924*I))

(-1.0*z**6 - z**5*(1.8 + 1.11022302462516e-16*I) + z**4*(1.10901234567901 - 2.22044604925031e-16*I) + 4.02666666666667*z**3 + z**2*(3.42220987654321 + 2.22044604925031e-16*I) + 0.27106172839506*z - 0.436296296296298)/(1.0*z**8 + 5.36296296296296*z**7 + 14.8350342935528*z**6 + z**5*(25.8620612711477 + 1.2490009027033e-16*I) + z**4*(30.9910123609206 + 3.23198258279768e-16*I) + z**3*(25.8620612711477 + 4.48970992985091e-16*I) + z**2*(14.8350342935528 + 3.14164484702989e-16*I) + z*(5.36296296296297 + 1.1808711936451e-16*I) + 1.0)

Generator return value: done
[0.512737016574588 + 1.25365208130776e-18*I 0.120244185253390
 -0.643701657458562 0.675044198895028 -0.288654696132599
 -0.200400165745853 0.474290502762428 -0.406824469060773]


In [17]:
Hdd = control.series(Hin, Hin_inv, Hfir, Hfir_inv)
display(Hdd)

TransferFunction(array([0.2 , 0.28, 1.01, 1.86, 1.01, 0.28, 0.2 , 0.  , 0.  ]), array([0.81 , 2.172, 3.096, 2.172, 0.81 , 0.   , 0.   , 0.   ]), 1)

In [18]:
# Hdd
zeros, poles, k = signal.tf2zpk(Hdd.num[0][0], Hdd.den[0][0])
zeros,poles = findzp(zeros, poles)
print(np.asarray(zeros))

print(np.asarray(poles))

pi = np.squeeze(np.argwhere(abs(np.asarray(poles)) <1), axis=1)
print(pi)
pint = [poles[i] for i in pi]
print(pint)

rdd = []
p = correlation_double_side(8)
while 1:
    try:
        rdd.append(next(p) / sigma2_x)
    except StopIteration as e:
        print('Generator return value:', e.value)
        break
print(np.real(np.asarray(rdd)))

[ 0.25+2.222j  0.25-2.222j -1.  +0.j    -1.  -0.j     0.05+0.444j
  0.05-0.444j]
[-0.741+0.828j -0.741-0.828j -0.6  +0.671j -0.6  -0.671j  0.   +0.j   ]
[2 3 4]
[(-0.5999999999999995+0.6708203932499363j), (-0.5999999999999995-0.6708203932499363j), 0j]


(-z + 0.0499999999999999 + 0.444409720865779*I)*(-z + 0.25 + 2.2220486043289*I)*(z - 0.25 + 2.2220486043289*I)*(z - 0.0499999999999999 + 0.444409720865779*I)*(z + 1.0 - 5.40191057691552e-9*I)*(z + 1.0 + 5.40191057691552e-9*I)/(z**2*(z + 0.6 + 0.670820393249936*I)*(z + 0.740740740740741 - 0.828173324999924*I)*(z + 0.740740740740741 + 0.828173324999924*I))

(-z + 0.0499999999999999 + 0.444409720865779*I)*(-z + 0.25 + 2.2220486043289*I)*(z - 0.25 + 2.2220486043289*I)*(z - 0.0499999999999999 + 0.444409720865779*I)*(z + 1.0 - 5.40191057691552e-9*I)*(z + 1.0 + 5.40191057691552e-9*I)/(z**2*(z + 0.6 - 0.670820393249936*I)*(z + 0.740740740740741 - 0.828173324999924*I)*(z + 0.740740740740741 + 0.828173324999924*I))

(2.0*z**9 + z**8*(9.44444444444444 + 5.55111512312578e-16*I) + z**7*(22.7975308641976 + 8.88178419700125e-16*I) + z**6*(33.7027407407408 + 2.66453525910038e-15*I) + z**5*(49.521987654321 + 3.5527136788005e-15*I) + z**4*(65.4307777777778 - 7.105427357601e-15*I) + z**3*(58.5674074074074 - 8.88178419700125e-15*I) + z**2*(28.0457530864198 - 2.66453525910038e-15*I) + z*(2.45530864197532 - 1.33226762955019e-15*I) - 1.28148148148148 + 1.11022302462516e-16*I)/(1.0*z**8 + 5.36296296296296*z**7 + 14.8350342935528*z**6 + z**5*(25.8620612711477 + 1.2490009027033e-16*I) + z**4*(30.9910123609206 + 3.23198258279768e-16*I) + z**3*(25.8620612711477 + 4.48970992985091e-16*I) + z**2*(14.8350342935528 + 3.14164484702989e-16*I) + z*(5.36296296296297 + 1.1808711936451e-16*I) + 1.0)

Generator return value: done
[0.734426519337016 + 3.50035785243652e-17*I -0.449313825796332
 -0.105107381215467 + 4.22684721803655e-17*I
 0.490073056353589 - 4.22684721803655e-17*I
 -0.502950688839778 - 1.05671180450914e-17*I
 0.206581650961327 + 4.22684721803655e-17*I
 0.159492076806627 - 4.22684721803655e-17*I -0.358721629446627]
